In [1]:
import psydac.core.interface as inter
import matplotlib.pyplot as plt

import numpy as np
import scipy.sparse as sparse

import derivatives as der
import projectors as proj
import evaluation as eva
import integrate as integr

In [9]:
# ... define boundary conditions in each direction (True: periodic, False: homogeneous Dirichlet)
bc = [False, False, False]
bc_x, bc_y, bc_z = bc

# ... length of computational domain in each direction
Lx = 1
Ly = 1
Lz = 1

# ... number of elements in each direction and corresponding element boundaries
Nel_x = 6
Nel_y = 6
Nel_z = 6

dx = Lx/Nel_x
dy = Ly/Nel_y
dz = Lz/Nel_z

el_b_x = np.linspace(0, Lx, Nel_x + 1)
el_b_y = np.linspace(0, Ly, Nel_y + 1)
el_b_z = np.linspace(0, Lz, Nel_z + 1)

# ... degree of spline basis (of the 0-forms) in each direction
p = [3, 3, 3]
px, py, pz = p

# ... number of basis functions
Nbase = [Nel_x + px, Nel_y + py, Nel_z + pz] 
Nbase_x, Nbase_y, Nbase_z = Nbase

# ... number of degrees of freedom
Nbase_x_0 = Nbase_x - bc_x*px - (1 - bc_x)*2
Nbase_y_0 = Nbase_y - bc_y*py - (1 - bc_y)*2
Nbase_z_0 = Nbase_z - bc_z*pz - (1 - bc_z)*2


# ... knot vectors
if bc_x == True:
    Tx = inter.make_periodic_knots(px, Nbase_x)
else:
    Tx = inter.make_open_knots(px, Nbase_x)
    
if bc_y == True:
    Ty = inter.make_periodic_knots(py, Nbase_y)
else:
    Ty = inter.make_open_knots(py, Nbase_y)
    
if bc_z == True:
    Tz = inter.make_periodic_knots(pz, Nbase_z)
else:
    Tz = inter.make_open_knots(pz, Nbase_z)
    
T = [Tx, Ty, Tz]

GRAD = der.GRAD_3d(p, Nbase, bc)
CURL = der.CURL_3d(p, Nbase, bc)
DIV  = der.DIV_3d(p, Nbase, bc)

In [10]:
# ... test projectors PI_0 and PI_1
if bc_x == True:
    Psi = lambda x, y, z : np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z)
    
    grad_Psi_x = lambda x, y, z: 2*np.pi*np.cos(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z)
    grad_Psi_y = lambda x, y, z: 2*np.pi*np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z)
    grad_Psi_z = lambda x, y, z: 2*np.pi*np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.cos(2*np.pi*z)
    
else:
    Psi = lambda x, y, z : x*(1 - x)*y*(1 - y)*z*(1 - z)

    grad_Psi_x = lambda x, y, z: (1 - 2*x)*y*(1 - y)*z*(1 - z)
    grad_Psi_y = lambda x, y, z: (1 - 2*y)*x*(1 - x)*z*(1 - z)
    grad_Psi_z = lambda x, y, z: (1 - 2*z)*x*(1 - x)*y*(1 - y)



x_test = np.random.rand(1)
y_test = np.random.rand(1)
z_test = np.random.rand(1)

xtest = [x_test, y_test, z_test]

print('Projector 0')
print(Psi(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V0(proj.PI_0(Psi, p, Nbase, T, bc), xtest, p, Nbase, T, bc)[0])

print('----------------------------')
print('Projector 1')
print(grad_Psi_z(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V1_z(proj.PI_1_z(grad_Psi_z, p, Nbase, T, bc), xtest, p, Nbase, T, bc)[0])

Projector 0
0.002768354417264192
0.0027683544172641917
----------------------------
Projector 1
-0.010558440324552196
-0.010558440324552196


In [4]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib

fig = plt.figure()
fig.set_figheight(8)
fig.set_figwidth(10)
ax = fig.add_subplot(111, projection='3d')


xplot = np.linspace(0, Lx, 101)
yplot = np.linspace(0, Ly, 101)
zplot = np.linspace(0, Lz, 101)

allplot = [xplot, yplot, zplot]

X, Y = np.meshgrid(xplot, yplot, indexing='ij')

Z = Psi(X, Y, zplot[20])
psi = eva.evaluate_field_V0(proj.PI_0(Psi, p, Nbase, T, bc), allplot, p, Nbase, T, bc)

psi = np.reshape(psi, (101, 101, 101))

Axes3D.plot_wireframe(ax, X, Y, Z, color='b')
Axes3D.plot_surface(ax, X, Y, psi[:, :, 20], color='r')

Using matplotlib backend: TkAgg


In [8]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib

fig = plt.figure()
fig.set_figheight(8)
fig.set_figwidth(10)
ax = fig.add_subplot(111, projection='3d')


xplot = np.linspace(0, Lx, 101)
yplot = np.linspace(0, Ly, 101)
zplot = np.linspace(0, Lz, 101)

allplot = [xplot, yplot, zplot]

X, Y = np.meshgrid(xplot, yplot, indexing='ij')

Z = grad_Psi_z(X, Y, zplot[20])
psi = eva.evaluate_field_V1_z(proj.PI_1_z(grad_Psi_z, p, Nbase, T, bc), allplot, p, Nbase, T, bc)

psi = np.reshape(psi, (101, 101, 101))

Axes3D.plot_wireframe(ax, X, Y, Z, color='b')
Axes3D.plot_surface(ax, X, Y, psi[:, :, 20], color='r')

Using matplotlib backend: TkAgg


In [11]:
# ... test projectors PI_1 and PI_2
if bc_x == True:
    Ex = lambda x, y, z : np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z)
    Ey = lambda x, y, z : np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.cos(2*np.pi*z)
    Ez = lambda x, y, z : np.cos(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z)
    
    curl_Ex = lambda x, y, z : 2*np.pi*(np.cos(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z) + np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z))
    curl_Ey = lambda x, y, z : 2*np.pi*(np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.cos(2*np.pi*z) + np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z))
    curl_Ez = lambda x, y, z : 2*np.pi*(np.cos(2*np.pi*x)*np.sin(2*np.pi*y)*np.cos(2*np.pi*z) + np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z))
    
else:
    Ex = lambda x, y, z: x*y*(1 - y)*z*(1 - z)
    Ey = lambda x, y, z: y*x*(1 - x)*z*(1 - z)
    Ez = lambda x, y, z: z*x*(1 - x)*y*(1 - y)

    curl_Ex = lambda x, y, z: z*x*(1 - x)*(1 - 2*y) - y*x*(1 - x)*(1 - 2*z)
    curl_Ey = lambda x, y, z: x*y*(1 - y)*(1 - 2*z) - z*(1 - 2*x)*y*(1 - y)
    curl_Ez = lambda x, y, z: y*(1 - 2*x)*z*(1 - z) - x*(1 - 2*y)*z*(1 - z)

x_test = np.random.rand(1)
y_test = np.random.rand(1)
z_test = np.random.rand(1)

xtest = [x_test, y_test, z_test]

print('Projector 1')
print(Ex(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V1_x(proj.PI_1_x(Ex, p, Nbase, T, bc), xtest, p, Nbase, T, bc)[0])


print('----------------------------')
print('Projector 2')
print(curl_Ez(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V2_z(proj.PI_2_z(curl_Ez, p, Nbase, T, bc), xtest, p, Nbase, T, bc)[0])

Projector 1
0.01188239626119957
0.011882396261199576
----------------------------
Projector 2
-0.07472199889748922
-0.07472199889748932


In [10]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib

fig = plt.figure()
fig.set_figheight(8)
fig.set_figwidth(10)
ax = fig.add_subplot(111, projection='3d')


xplot = np.linspace(0, Lx, 101)
yplot = np.linspace(0, Ly, 101)
zplot = np.linspace(0, Lz, 101)

allplot = [xplot, yplot, zplot]

X, Y = np.meshgrid(xplot, yplot, indexing='ij')

Z = curl_Ez(X, Y, zplot[20])
psi = eva.evaluate_field_V2_z(proj.PI_2_z(curl_Ez, p, Nbase, T, bc), allplot, p, Nbase, T, bc)

psi = np.reshape(psi, (101, 101, 101))

Axes3D.plot_wireframe(ax, X, Y, Z, color='b')
Axes3D.plot_surface(ax, X, Y, psi[:, :, 20], color='r')

Using matplotlib backend: TkAgg


In [12]:
# ... test projectors PI_2 and PI_3
if bc_x == True:
    Bx = lambda x, y, z : np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z)
    By = lambda x, y, z : np.sin(2*np.pi*x)*np.sin(2*np.pi*y)*np.cos(2*np.pi*z)
    Bz = lambda x, y, z : np.cos(2*np.pi*x)*np.sin(2*np.pi*y)*np.sin(2*np.pi*z)
    
    div_B = lambda x, y, z : 2*np.pi*(np.cos(2*np.pi*x)*np.cos(2*np.pi*y)*np.sin(2*np.pi*z) + np.sin(2*np.pi*x)*np.cos(2*np.pi*y)*np.cos(2*np.pi*z) + np.cos(2*np.pi*x)*np.sin(2*np.pi*y)*np.cos(2*np.pi*z))

else:
    Bx = lambda x, y, z : x*(1 - x)*y*z
    By = lambda x, y, z : y*(1 - y)*x*z
    Bz = lambda x, y, z : z*(1 - z)*x*y
    
    div_B = lambda x, y, z : y*z + x*z + x*y - 6*x*y*z
    
x_test = np.random.rand(1)
y_test = np.random.rand(1)
z_test = np.random.rand(1)

xtest = [x_test, y_test, z_test]

print('Projector 2')
print(Bx(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V2_x(proj.PI_2_x(Bx, p, Nbase, T, bc), xtest, p, Nbase, T, bc)[0])

print('----------------------------')
print('Projector 3')
print(div_B(x_test, y_test, z_test)[0])
print(eva.evaluate_field_V3(proj.PI_3(div_B, p, Nbase, T, bc), xtest, p, Nbase, T, bc)[0])

Projector 2
0.03891840038961674
0.03891840038961674
----------------------------
Projector 3
-0.5905875998969077
-0.5905875998969072


In [8]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib

fig = plt.figure()
fig.set_figheight(8)
fig.set_figwidth(10)
ax = fig.add_subplot(111, projection='3d')


xplot = np.linspace(0, Lx, 101)
yplot = np.linspace(0, Ly, 101)
zplot = np.linspace(0, Lz, 101)

allplot = [xplot, yplot, zplot]

X, Y = np.meshgrid(xplot, yplot, indexing='ij')

Z = div_B(X, Y, zplot[20])
psi = eva.evaluate_field_V3(proj.PI_3(div_B, p, Nbase, T, bc), allplot, p, Nbase, T, bc)

psi = np.reshape(psi, (101, 101, 101))

Axes3D.plot_wireframe(ax, X, Y, Z, color='b')
Axes3D.plot_surface(ax, X, Y, psi[:, :, 20], color='r')

Using matplotlib backend: TkAgg


In [13]:
# ... check commuting diagram property (V0 --> V1)
vec_0 = GRAD.dot(proj.PI_0(Psi, p, Nbase, T, bc))

vec_1_x = proj.PI_1_x(grad_Psi_x, p, Nbase, T, bc)
vec_1_y = proj.PI_1_y(grad_Psi_y, p, Nbase, T, bc)
vec_1_z = proj.PI_1_z(grad_Psi_z, p, Nbase, T, bc)

vec_1 = np.append(vec_1_x, np.append(vec_1_y, vec_1_z))

print('max. error :', np.abs(vec_0 - vec_1).max())

max. error : 5.2909066017292616e-17


In [14]:
# check commuting diagram property (V1 --> V2)
vec_1_x = proj.PI_1_x(Ex, p, Nbase, T, bc)
vec_1_y = proj.PI_1_y(Ey, p, Nbase, T, bc)
vec_1_z = proj.PI_1_z(Ez, p, Nbase, T, bc)

vec_1 = CURL.dot(np.append(vec_1_x, np.append(vec_1_y, vec_1_z)))

vec_2_x = proj.PI_2_x(curl_Ex, p, Nbase, T, bc)
vec_2_y = proj.PI_2_y(curl_Ey, p, Nbase, T, bc)
vec_2_z = proj.PI_2_z(curl_Ez, p, Nbase, T, bc)

vec_2 = np.append(vec_2_x, np.append(vec_2_y, vec_2_z))

print('max. error :', np.abs(vec_1 - vec_2).max())

max. error : 6.201636426617085e-17


In [15]:
# check commuting diagram property (V2 --> V3)
vec_2_x = proj.PI_2_x(Bx, p, Nbase, T, bc)
vec_2_y = proj.PI_2_y(By, p, Nbase, T, bc)
vec_2_z = proj.PI_2_z(Bz, p, Nbase, T, bc)

vec_2 = DIV.dot(np.append(vec_2_x, np.append(vec_2_y, vec_2_z)))

vec_3 = proj.PI_3(div_B, p, Nbase, T, bc)

print('max. error :', np.abs(vec_2 - vec_3).max())

max. error : 1.474514954580286e-17
